In [1]:
import torch
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
from transformers import AutoTokenizer, PreTrainedTokenizerFast
from dictionary_learning import AutoEncoder
from nnsight import LanguageModel
from dictionary_learning.utils import read_csv
from IPython.display import display, HTML

In [2]:
import scipy.sparse
from scipy.sparse import csr_matrix
from dictionary_learning.sparse_feature_writer import SparseFeatureWriter

import gc
import h5py
from collections import defaultdict

In [4]:
# load autoencoder
ae = AutoEncoder.from_pretrained("/gpfs/helios/home/jpauklin/dictionary_learning/saes/estMedSae120425/trainer_0/ae_12042025.pt").to("cuda") # to is rquired to load to GPU

device = "cuda:0" #GPU
# Load Model
model_name = "/gpfs/space/projects/stacc_health/gpt2_model/estMed-gpt2_fine_tuned4/estMed-gpt2_fine_tuned4"
model = LanguageModel(
    model_name,
    device_map=device,
)

activation_dim = model.transformer.h[0].ln_1.normalized_shape[0] # output dimension of the MLP = 768

/gpfs/helios/home/jpauklin/dictionary_learning/dictionary_learning/dictionary.py:132: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = t.load(path)


In [5]:
data = read_csv("/gpfs/space/projects/stacc_health/data-synthetic/100k_synthetic_texts.csv", 100) # (csv_path, nr_of_text_batches_to_read)
text = list(data) # list from iterator

In [5]:
#text.append("See on test lause, mis on käsitsi lisatud.")

In [6]:
tokenizer_filepath = "/gpfs/space/projects/stacc_health/gpt2_model/estMed-gpt2_fine_tuned4/estMed-gpt2_fine_tuned4/tokenizer.json"
tokenizer = PreTrainedTokenizerFast(tokenizer_file=tokenizer_filepath)
tokenizer.pad_token = "<pad>"

# Padding to have batched tensors with the same length.    
tokens = tokenizer(text, return_tensors="pt", padding=True)   # , truncation=True


In [7]:
"""
Idee
Ükshaaval jälgida aktivatsioone. anda sisend üks batch haaval ette. Salvestada features (1 batchi kohta korraga) faili. 

Siht. Meil on kogu andmete kohta featurid. Saame otsida tugevaid aktivatsioone üle kõigi tokenite. 
Saame kindla featuri kõik tugevaimad aktivatsioonid. 

Mis faili formaat? txt või csv 
Failis on batchid eraldatud üksteisest ridade kaupa.

Näiteks leitakse tugev aktivatsioon ühes batchis. Lihtsaim viis teha, et terve batch loetakse ja ss kui on akt 
salvestatakse batch tulemusse. 
"""

'\nIdee\nÜkshaaval jälgida aktivatsioone. anda sisend üks batch haaval ette. Salvestada features (1 batchi kohta korraga) faili. \n\nSiht. Meil on kogu andmete kohta featurid. Saame otsida tugevaid aktivatsioone üle kõigi tokenite. \nSaame kindla featuri kõik tugevaimad aktivatsioonid. \n\nMis faili formaat? txt või csv \nFailis on batchid eraldatud üksteisest ridade kaupa.\n\nNäiteks leitakse tugev aktivatsioon ühes batchis. Lihtsaim viis teha, et terve batch loetakse ja ss kui on akt \nsalvestatakse batch tulemusse. \n'

In [3]:
writer = SparseFeatureWriter("features/features100.h5")

h5 file already exists. Clear current data? (y/n) y


In [7]:
for input_ids, attention_mask in zip(tokens['input_ids'], tokens.get('attention_mask')):
    print(input_ids.size())
    
    with model.trace(input_ids, attention_mask) as tracer:
    
        # Selecting a specific layer to capture
        mlp_output = model.transformer.h[11].mlp.c_proj.output.save()
        # The model is actually run upon exiting the tracing context. (https://nnsight.net/notebooks/tutorials/walkthrough/)
    
    activations = mlp_output.value
    features = ae.encode(activations)[0] # for each token a list of feature activations
    
    csr_batch = csr_matrix(features.detach().cpu().numpy()) # Compressed Sparse Row matrix from features (tensor -> numpy array)
    writer.append(csr_batch, input_ids)

    # Free up space
    torch.cuda.empty_cache() 
    gc.collect()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


torch.Size([512])


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size

In [17]:
j = 1
for batch in tokens['input_ids']:
    print()
    print(f"batch {j}")
    for token in batch:
        decoded_token = tokenizer.decode(token)
        print(decoded_token, end=" ")
    j += 1


batch 1
NAME  :  läheb  pik ale  lennureisi le .  seal  tunneb ,  et  ja bajal ad  valutavad ,  kurk  on  kuiv  köha .  obj .  kops  puhas ,  neel  punetav  DATE  NAME  :  ta  pereõe  käeline  tegevus .  veenivere  võtmine  vaakumsüsteemiga .  analüüside  teostamine -  automaatuuring ,  crp . analüüside  vastused kliinilise  keemia  uuringud  DATE  :  crp  <  DATE  mg / l  [ norm .  -  10 ]  DATE  :  antistreptolüsiin - o  117 , 8  iu / ml  [ norm .  -  200 ] <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <

objektiivne  leid :  DATE  gks  -  0 / 0 ; <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

-  pt  kaebab  valudele  kaelas ,  kuklas ,  peavalud ,  kohin  kõrvades . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <

adj  üldseisundis  kergelt  ülekaalus  noore ma  kehaehitusega  noormees .  ta  lööbeta .  kehakaal  36 , 8 kg ,  pikkus  127 cm .  . kmi  18 , 4 .  südametoonid  regulaarsed ,  fr . 76 x ´.  rr  104 / 64 mmhg .  pulsid  randmetel  palpeeritavad .  kilpnääre  ii . <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pa

-  kn  kontroll <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> 

In [18]:
with h5py.File("features/features100.h5", 'r') as f:
    shape = f.attrs['shape']
    csr = csr_matrix((f['data'][:], f['indices'][:], f['indptr'][:]), shape=shape)
    tokens_h5 = f['tokens'][:]
    
    i = 0
    for token in tokens_h5:
        if i % 512 == 0:
            print()
            print(f"batch {i / 512}")
        decoded_token = tokenizer.decode(token)
        print(decoded_token, end=" ")
        i += 1



batch 0.0
NAME  :  läheb  pik ale  lennureisi le .  seal  tunneb ,  et  ja bajal ad  valutavad ,  kurk  on  kuiv  köha .  obj .  kops  puhas ,  neel  punetav  DATE  NAME  :  ta  pereõe  käeline  tegevus .  veenivere  võtmine  vaakumsüsteemiga .  analüüside  teostamine -  automaatuuring ,  crp . analüüside  vastused kliinilise  keemia  uuringud  DATE  :  crp  <  DATE  mg / l  [ norm .  -  10 ]  DATE  :  antistreptolüsiin - o  117 , 8  iu / ml  [ norm .  -  200 ] <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

-  saatekiri  dr .  ta patsiendil  progresseeruv  suhkruhaigus ,  tablettravil ,  hüpertoonik ,  läbipõetud  infarkt ,  korduvad  rasked  insul did ,  täna  hommikul  30  min .  veresuhkru  väärtuste  kõrgenemine  DATE  mmol / l ,  käis  endokrinoloogi  vastuvõtul  DATE  perh - s  ( kus  jäi  patsiendi  sõnul  kõik  uuringud  pooleli ) <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <p

patsiendil  uuringud  käigus  -  südame ultraheli :  tromb ita ,  vähene  parema  niudearteri  stenoos .  uuritud  st  erineva  süsteem süsteemi de  ar et rite  osas .  ct  angiograafial  kirjeldatud  parema  a .  iliaca  externa  lõpposa  anom aal iat ,  kuid  selle  sees  venoosses  faasis  a .  iliaca  externa  alg osas  endiselt  lisakude . saadud  avast ust  kinnitab  kliiniku  ja  siiski  arvest atav ate  haiguste  protok ol lid .  DATE  teostatud  parema  reiearteri  pta  ja  stentimine .  protseduuri  järgne  periood  tavapärane .  DATE  hemogramm  wbc  7 . 96  ( 3 , 5 .  8 , 8  e 9 / l  )  rbc  4 . 41  ( 4 , 2 .  5 , 7  e 12 / l  )  hgb  129  ( 134 .  170  g / l  )  hct  38  ( 40 .  50  %  )  mcv  86 . 1  ( 82 .  98  fl  )  mch  29 . 2  ( 27 .  33  pg  )  mchc  339  ( 317 .  357  g / l  )  plt  246  ( 145 .  390  e 9 / l  )  rdw - cv  13 . 1  ( 11 , 6 .  14  %  )  s , p - uurea  6 . 1  (< 8 . 3  mmol / l  )  s , p - crp  1  (< 5  mg / l  )  s , p - na  139  ( 136 .  145  mmol 

NAME  :  silmarõhu  mõõtmine <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad> <pad>

In [9]:
# Using nnsight, we hook into one of the MLP's layer and inspect them during a forward pass.
# This gets us the layer's activations (output of c_proj)

# The "with" keyword starts a context-like object. It defines that we will run the model with tracing. 
with model.trace(tokens['input_ids'], tokens.get('attention_mask')) as tracer:
    
    # Selecting a specific layer to capture
    mlp_output = model.transformer.h[11].mlp.c_proj.output.save()
    
# The model is actually run upon exiting the tracing context. (https://nnsight.net/notebooks/tutorials/walkthrough/)
activations = mlp_output.value  # shape: (batch_size aka nr_of_inputs, seq_len , hidden_dim_activation_value)

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


In [10]:
features = ae.encode(activations) # get features from activations
reconstructed_activations = ae.decode(features)

In [ ]:
activations

In [ ]:
reconstructed_activations

In [ ]:
features.size()

In [ ]:
type(features[0][0][0])

In [19]:
with h5py.File("features/featuresWithTokens.h5", 'r') as f:
    shape = f.attrs['shape']
    csr = csr_matrix((f['data'][:], f['indices'][:], f['indptr'][:]), shape=shape)
    tokens_h5 = f['tokens'][:]
    
print(tokens_h5[0])

321


In [20]:
with h5py.File("features/featuresWithTokens.h5", 'r') as f:
    shape = f.attrs['shape']
    csr = csr_matrix((f['data'][:], f['indices'][:], f['indptr'][:]), shape=shape)
    tokens_h5 = f['tokens'][:]

    for i, token in enumerate(tokens_h5[:20]):
        row = csr.getrow(i)
        if row.nnz:
            topk_idx = row.indices[row.data.argsort()[::-1][:3]]
            topk_val = row.data[row.data.argsort()[::-1][:3]]
            print(f"{token} → features {topk_idx.tolist()} = {topk_val.tolist()}")

321 → features [3401, 10404, 6611] = [28.051488876342773, 19.718618392944336, 8.891932487487793]
322 → features [3401, 8914, 10404] = [2.2227442264556885, 2.055420398712158, 2.0495247840881348]
2469 → features [2492, 6017, 1315] = [1.4795663356781006, 1.404335379600525, 1.319575309753418]
773 → features [2492, 3155, 1880] = [2.208782434463501, 1.1387426853179932, 1.0914661884307861]
544 → features [2492, 3155, 8914] = [2.407727003097534, 1.726718544960022, 1.6081944704055786]
35199 → features [10684, 2492, 1315] = [1.526919960975647, 1.3731974363327026, 1.0258238315582275]
275 → features [1315, 4043, 10125] = [1.9891438484191895, 0.9500616788864136, 0.7149729132652283]
14 → features [2492, 1315, 8914] = [1.8179043531417847, 1.2704567909240723, 1.0956710577011108]
4686 → features [2492, 4271, 8914] = [1.0401057004928589, 1.008610486984253, 0.9963252544403076]
2022 → features [1315, 2492, 7420] = [1.5138278007507324, 1.447295904159546, 0.9973737001419067]
12 → features [1315, 2492, 3155]

In [ ]:
def get_batch(batch_idx):
    token_batch_size = 462
    
    with h5py.File("features/featuresWithTokens.h5", 'r') as f:
        shape = f.attrs['shape']
        csr = csr_matrix((f['data'][:], f['indices'][:], f['indptr'][:]), shape=shape)
        tokens_h5 = f['tokens'][:]
        
    row = csr.getrow(i)
    
    feature_activations_for_seq =  csr
    
    return 

In [ ]:

def token_top_features(tokens, features, batch_idx):
    # features shape: (batch_size, seq_len, num_features)
    feature_activations_for_seq = features[batch_idx]

    # Get corresponding tokens
    input_tokens = tokens['input_ids'][batch_idx]

    # For each token
    for token_idx, token in enumerate(input_tokens):
        decoded_token = tokenizer.decode(token)
        print(f"Token: {decoded_token}")
        feature_acts = feature_activations_for_seq[token_idx]

        # Show top k most active features for token
        topk = torch.topk(feature_acts, k=5)
        print(f"  Top features: {topk.indices.tolist()} with activations {topk.values.tolist()}")

In [ ]:
token_top_features(tokens, features, 0)

In [ ]:
# TODO: Funktsioon, mis leiab harva (kuid tugevasti) käivituvad feature'id.

In [ ]:
# given a specific feature and a batch of input text, outputs the feature's activation value for each token.

def feature_activations(tokens, features, batch_idx, feature_idx):
    # features shape: (batch_size, seq_len, num_features)
    feature_activations_for_seq = features[batch_idx]  # shape: (seq_len, num_features)

    # Get corresponding tokens
    input_tokens = tokens['input_ids'][batch_idx]

    # Print given feature's activation value for each token
    print(f"Feature: {feature_idx}")
    for token_idx, token in enumerate(input_tokens):
        decoded_token = tokenizer.decode(token)
        #token_clean = decoded_token.replace("Ġ", "")
        feature_acts = features[batch_idx][token_idx]
        print(f"{decoded_token: >20} = {feature_acts[feature_idx]}")

In [ ]:
feature_activations(tokens, features, 0, 2492)

In [ ]:
def visualize_feature_activations(tokens, features, batch_idx, feature_idx):
    # detach from computation graph. Move to CPU memory, since numpy only works on CPU tensors.
    feature_activations = features[batch_idx, : , feature_idx].detach().cpu().numpy()

    # Get corresponding tokens
    input_tokens = tokens['input_ids'][batch_idx]
    
    # Color mapping
    max_val = 30 
    min_val = 0
    forward = lambda x: x** (1/3) # function. Transforms the data values before applying the colormap.
    inverse = lambda x: x**3 # function. Transforms from the normalized scale [0, 1] back to the original data space.
    norm = matplotlib.colors.FuncNorm((forward, inverse), vmin=min_val, vmax=max_val) #(https://matplotlib.org/stable/users/explain/colors/colormapnorms.html)
    #norm = matplotlib.colors.Normalize(vmin=min_val, vmax=max_val)
    cmap = matplotlib.colormaps.get_cmap('Oranges')

    # Build HTML with color highlighting
    html = ""
    for token, activation in zip(input_tokens, feature_activations): 
        decoded_token = tokenizer.decode(token)
     
        # TODO: Ei lisaks spani padding tokenite korral
    
        color = matplotlib.colors.rgb2hex(cmap(norm(activation)))
        html += f'<span style="background-color:{color}; padding: 2px; margin: 2px; border-radius: 3px;">{decoded_token}</span> '

    display(HTML(html))

In [ ]:
visualize_feature_activations(tokens, features, 0, 10684)

In [ ]:
def find_rarely_active_features(h5_path, threshold=0.1, max_occurrences=5):
    """
    Find features that rarely have strong activations in an HDF5 file.

    Parameters:
        h5_path (str): Path to the HDF5 file.
        threshold (float): Minimum activation to count as 'strong'.
        max_occurrences (int): Maximum allowed number of times a feature may exceed the threshold.
    """
    with h5py.File(h5_path, 'r') as f:
        # Reconstruct CSR matrix
        shape = f.attrs['shape']
        features = csr_matrix((f['data'][:], f['indices'][:], f['indptr'][:]), shape=shape)
        tokens = f['tokens'][:]
    
    num_features = features.shape[1]
    activation_counts = np.zeros(num_features, dtype=int)
    token_tracker = defaultdict(list)

    # Iterate over rows (each token)
    for row_idx in range(features.shape[0]):
        row = features.getrow(row_idx)
        strong_mask = row.data > threshold
        strong_indices = row.indices[strong_mask]

        for feature_idx in strong_indices:
            activation_counts[feature_idx] += 1
            if len(token_tracker[feature_idx]) < max_occurrences:
                token_tracker[feature_idx].append(tokens[row_idx].decode('utf-8') if isinstance(tokens[row_idx], bytes) else tokens[row_idx])

    # Identify rarely active features
    rarely_active = np.where(activation_counts < max_occurrences)[0]

    print(f"\n📉 Found {len(rarely_active)} rarely active features (threshold > {threshold}, occurred < {max_occurrences} times):\n")

    for feature_idx in rarely_active:
        example_tokens = token_tracker[feature_idx]
        print(f"Feature {feature_idx} was strongly activated {activation_counts[feature_idx]} times.")
        print(f"  Example tokens: {example_tokens}\n")

In [ ]:
# Note: Teha suur tulemustabel/fail, kus saab otsida, feature'eid, mis harva aktiveeruvad või tokenite järgi (terviseterminid)